In [17]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint, uniform
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

/home/mateo/MELI/meli-mlops-mateo-restrepo/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
#Carga de datos
df = pd.read_csv('/home/mateo/MELI/meli-mlops-mateo-restrepo/data/raw/HousingData.csv')
variable_objetivo = 'MEDV'
variable_categoricas = ['CHAS', 'RAD']

# Manejo de nulos
if df.isnull().sum().any():
    print("Se detectaron valores nulos")
    for col in df.columns:
        if df[col].isnull().any():
            # Estrategia 1: Imputar con la MODA para variables categóricas
            if col in variable_categoricas:
                mode_val = df[col].mode()[0]
                df[col] = df[col].fillna(mode_val)
                print(f"   - Columna '{col}': Nulos rellenados con la MODA ({mode_val})")
            
            # Estrategia 2: Imputar con la MEDIA para el resto (numéricas)
            else: 
                mean_val = df[col].mean()
                df[col] = df[col].fillna(mean_val)
                print(f"   - Columna '{col}': Nulos rellenados con la MEDIA ({mean_val})")
else:
    print("No se encontraron valores nulos.")

# Eliminar duplicados
len_original = len(df)
df.drop_duplicates(inplace=True)
if len(df) < len_original:
    print(f"Se eliminaron {len_original - len(df)} filas duplicadas.")
else:
    print("No se encontraron filas duplicadas.")

df_limpio = df

df_limpio.describe()

Se detectaron valores nulos
   - Columna 'CRIM': Nulos rellenados con la MEDIA (3.6118739711934156)
   - Columna 'ZN': Nulos rellenados con la MEDIA (11.2119341563786)
   - Columna 'INDUS': Nulos rellenados con la MEDIA (11.083991769547325)
   - Columna 'CHAS': Nulos rellenados con la MODA (0.0)
   - Columna 'AGE': Nulos rellenados con la MEDIA (68.51851851851852)
   - Columna 'LSTAT': Nulos rellenados con la MEDIA (12.715432098765433)
No se encontraron filas duplicadas.


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.611874,11.211934,11.083992,0.067194,0.554695,6.284634,68.518519,3.795043,9.549407,408.237154,18.455534,356.674032,12.715432,22.532806
std,8.545770,22.921051,6.699165,0.250605,0.115878,0.702617,27.439466,2.105710,8.707259,168.537116,2.164946,91.294864,7.012739,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.083235,0.000000,5.190000,0.000000,0.449000,5.885500,45.925000,2.100175,4.000000,279.000000,17.400000,375.377500,7.230000,17.025000
50%,0.290250,0.000000,9.900000,0.000000,0.538000,6.208500,74.450000,3.207450,5.000000,330.000000,19.050000,391.440000,11.995000,21.200000
75%,3.611874,11.211934,18.100000,0.000000,0.624000,6.623500,93.575000,5.188425,24.000000,666.000000,20.200000,396.225000,16.570000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [15]:
# División de datos
X = df_limpio.drop(columns=['MEDV'])
y = df_limpio['MEDV']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

print(f"   - Entrenamiento: {X_train.shape[0]} muestras, {X_train.shape[1]} características")
print(f"   - Prueba: {X_test.shape[0]} muestras, {X_test.shape[1]} características")


   - Entrenamiento: 404 muestras, 13 características
   - Prueba: 102 muestras, 13 características


In [16]:
# Escalado de Variables 

scaler = StandardScaler()
X_train_scaled_np = scaler.fit_transform(X_train)
X_test_scaled_np = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled_np, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled_np, columns=X_test.columns, index=X_test.index)
print("Características escaladas correctamente.")

Características escaladas correctamente.


In [19]:
def evaluar_modelo(modelo, X_train, y_train, X_test, y_test, nombre_experimento=""):
    print(f"\n--- Evaluando: {nombre_experimento} ---")
    y_train_pred = modelo.predict(X_train)
    y_test_pred = modelo.predict(X_test)
    
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)
    
    print(f"📊 Test  -> RMSE: {rmse_test:.4f} | R²: {r2_test:.4f}")
    return {"Experimento": nombre_experimento, "RMSE_Test": rmse_test, "R2_Test": r2_test}


In [20]:
# Lista para almacenar los resultados
results_list = []

#Experimento 1: Búsqueda de Hiperparámetros (Todas las Características) ---

print("EXPERIMENTO 1: Búsqueda de Hiperparámetros (Todas las Características)")

param_dist = {"n_estimators": randint(50, 500),
               "max_depth": randint(3, 15), 
               "learning_rate": uniform(0.01, 0.3), 
               "subsample": uniform(0.6, 0.4), 
               "colsample_bytree": uniform(0.6, 0.4)
               }
random_search = RandomizedSearchCV(xgb.XGBRegressor(random_state=42),
                                   param_distributions=param_dist,
                                   n_iter=25, 
                                   cv=5, 
                                   scoring='neg_mean_squared_error', 
                                   random_state=42,
                                   n_jobs=-1, 
                                   verbose=0)


random_search.fit(X_train_scaled, y_train)
best_model_exp1 = random_search.best_estimator_
print(f"Mejores Hiperparámetros Encontrados: {random_search.best_params_}")
results_list.append(evaluar_modelo(best_model_exp1, X_train_scaled, y_train, X_test_scaled, y_test, "Exp 1: Optimizado (Todas las Features)"))

EXPERIMENTO 1: Búsqueda de Hiperparámetros (Todas las Características)
Mejores Hiperparámetros Encontrados: {'colsample_bytree': np.float64(0.7098887171960256), 'learning_rate': np.float64(0.17837302775431035), 'max_depth': 5, 'n_estimators': 150, 'subsample': np.float64(0.9886848381556415)}

--- Evaluando: Exp 1: Optimizado (Todas las Features) ---
📊 Test  -> RMSE: 2.8315 | R²: 0.8907


In [22]:
print("EXPERIMENTO 2: Selección de Características con SHAP")

# Paso 1: Entrenar un modelo base para calcular los valores SHAP
print("Paso 1: Entrenando modelo selector...")
selector_model = xgb.XGBRegressor(random_state=42)
selector_model.fit(X_train_scaled, y_train)

# Paso 2: Calcular los valores SHAP
print("Paso 2: Calculando valores SHAP...")
explainer = shap.TreeExplainer(selector_model)
shap_values = explainer.shap_values(X_train_scaled)
mean_abs_shap = np.abs(shap_values).mean(axis=0)

# Crear un DataFrame para visualizar la importancia
shap_importance_df = pd.DataFrame({
    'feature': X_train_scaled.columns,
    'shap_importance': mean_abs_shap
}).sort_values('shap_importance', ascending=False)

print("📊Importancia de características según SHAP:")
print(shap_importance_df)

# Paso 3: Seleccionar características con SHAP > umbral (percentil 20)
print("Paso 3: Seleccionando características...")
threshold = np.percentile(mean_abs_shap, 20)
selected_features_mask = mean_abs_shap >= threshold
selected_features = X_train_scaled.columns[selected_features_mask].tolist()

print(f"Se seleccionaron {len(selected_features)}/{len(X.columns)} características (SHAP > percentil 20 = {threshold:.4f}):")
print(selected_features)

# Crear nuevos conjuntos de datos con solo las características seleccionadas
X_train_selected = X_train_scaled[selected_features]
X_test_selected = X_test_scaled[selected_features]

# Paso 4: Entrenar un nuevo modelo (con parámetros por defecto) en los datos seleccionados
print("Paso 4: Entrenando modelo final con características seleccionadas...")
model_selected_features = xgb.XGBRegressor(random_state=42)
model_selected_features.fit(X_train_selected, y_train)
results_list.append(evaluar_modelo(model_selected_features, X_train_selected, y_train, X_test_selected, y_test, "Exp 2: Features Seleccionadas con SHAP (Default)"))

EXPERIMENTO 2: Selección de Características con SHAP
Paso 1: Entrenando modelo selector...
Paso 2: Calculando valores SHAP...
📊Importancia de características según SHAP:
    feature  shap_importance
5        RM         3.844686
12    LSTAT         2.577572
0      CRIM         0.756323
10  PTRATIO         0.654981
7       DIS         0.620467
6       AGE         0.554832
4       NOX         0.383089
9       TAX         0.352447
11        B         0.253588
8       RAD         0.166307
2     INDUS         0.153890
3      CHAS         0.057108
1        ZN         0.050280
Paso 3: Seleccionando características...
Se seleccionaron 10/13 características (SHAP > percentil 20 = 0.1589):
['CRIM', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
Paso 4: Entrenando modelo final con características seleccionadas...

--- Evaluando: Exp 2: Features Seleccionadas con SHAP (Default) ---
📊 Test  -> RMSE: 2.4640 | R²: 0.9172


In [23]:
# Experimento 3: Búsqueda de Hiperparámetros (Características Seleccionadas por SHAP)
print("EXPERIMENTO 3: Búsqueda de Hiperparámetros (Características Seleccionadas por SHAP)")

random_search_selected = RandomizedSearchCV(xgb.XGBRegressor(random_state=42),
                                            param_distributions=param_dist,
                                            n_iter=25, cv=5,
                                            scoring='neg_mean_squared_error',
                                            random_state=42,
                                            n_jobs=-1, 
                                            verbose=0)

random_search_selected.fit(X_train_selected, y_train)

best_model_exp3 = random_search_selected.best_estimator_

print(f"Mejores Hiperparámetros Encontrados: {random_search_selected.best_params_}")

results_list.append(evaluar_modelo(best_model_exp3, X_train_selected, y_train, X_test_selected, y_test, "Exp 3: Optimizado (Features Seleccionadas por SHAP)"))

EXPERIMENTO 3: Búsqueda de Hiperparámetros (Características Seleccionadas por SHAP)
Mejores Hiperparámetros Encontrados: {'colsample_bytree': np.float64(0.7427013306774357), 'learning_rate': np.float64(0.09428035290621423), 'max_depth': 14, 'n_estimators': 90, 'subsample': np.float64(0.718509402281633)}

--- Evaluando: Exp 3: Optimizado (Features Seleccionadas por SHAP) ---
📊 Test  -> RMSE: 2.7612 | R²: 0.8960


In [25]:
print("TABLA COMPARATIVA DE RESULTADOS")
results_df = pd.DataFrame(results_list)
print(results_df.to_string(index=False))

TABLA COMPARATIVA DE RESULTADOS
                                        Experimento  RMSE_Test  R2_Test
             Exp 1: Optimizado (Todas las Features)   2.831456 0.890676
   Exp 2: Features Seleccionadas con SHAP (Default)   2.463976 0.917212
Exp 3: Optimizado (Features Seleccionadas por SHAP)   2.761163 0.896037
